## Import dependencies

In [0]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

## Setup notebook parameters

In [0]:
# Train parameters
INIT_LR = 1e-4
EPOCHS  = 20
BS      = 32

# Data parameters
data_path = '/content/drive/My Drive/covid-19/dataset/'

## Load data

In [0]:
def load_image_paths():
  file_paths_and_labels = []
  subfolders            = [x[0] for x in os.walk(data_path)][1:]
  
  for folder in subfolders:
    label = folder.split('/')[-1]
    
    [file_paths_and_labels.append((folder + '/' + file_name, label)) for file_name in os.listdir(folder)]

  return file_paths_and_labels

def load_images():
  file_paths_and_labels = load_image_paths()
  data   = []
  labels = []
  
  total_images = len(file_paths_and_labels)

  for i, (image_path, label) in enumerate(file_paths_and_labels):
    print("Loading images: %.2f%%" % (100*(i+1)/total_images))

    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(label)
  
  return data, labels

data, labels = load_images()

## One-Hot encoding and Data augmentation

In [0]:
label_binarizer = LabelBinarizer()
labels          = label_binarizer.fit_transform(labels)
labels          = to_categorical(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)

aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

## Load Model

In [0]:
head_model = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

base_model = head_model.output
base_model = AveragePooling2D(pool_size=(7, 7))(base_model)
base_model = Flatten(name="flatten")(base_model)
base_model = Dense(128, activation="relu")(base_model)
base_model = Dropout(0.5)(base_model)
base_model = Dense(2, activation="softmax")(base_model)

model = Model(inputs=head_model.input, outputs=base_model)

for layer in head_model.layers:
	layer.trainable = False

## Compile and train model head

In [0]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

## Model validation And Save

In [0]:
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=label_binarizer.classes_))

model.save('image_net_model_trained', save_format="h5")